# Sentence BERT

* https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/nlp/text_embedding(japanese)/sentencebert.ipynb)

In [1]:
%%capture
!pip install transformers
!pip install fugashi
!pip install ipadic

In [ ]:
from transformers import BertJapaneseTokenizer, BertModel
import torch

#### Model

In [ ]:
class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)

model = SentenceBertJapanese("sonoisa/sentence-bert-base-ja-mean-tokens-v2")

#### Data Set Preparation

In [ ]:
text = [
    '機械学習の勉強をする',
    'あいうえお',
    '量子コンピュータは量子力学の原理を応用したコンピュータ'
]

In [ ]:
embeddings = model.encode(text, batch_size=8)

In [ ]:
embeddings